# <center> State preparation tutorial [[Link]](https://pennylane.readthedocs.io/en/user-docs-refactor/tutorials/pennylane_run_state_preparation.html#state-preparation) </center>

Build and optimize a circuit to prepare arbitrary single-qubit states, including mixed states. <br>
Along the way, we also show how to:

1. Construct compact expressions for circuits compoed of many layers.
2. Succinctly evaluate expectation values of many observables.
3. Estimate expectation values from repeated measurements, as in real hardware.

Density matrix of a qubit can be uniquely described in terms of its three-dimensional Bloch vector $\vec{a}=(a_x,a_y,a_z)$ as:
$$
\hat{\rho} = \frac{1}{2}(1+a_x\hat{\sigma}_x + a_y\hat{\sigma}_y + a_z\hat{\sigma}_z),
$$
where $\hat{\sigma}_x$, $\hat{\sigma}_y$, $\hat{\sigma}_z$ are the Pauli matrices. <br>
Any Bloch vector corresponds to a valid density matrix as long as $||\vec{a}||\le 1$.

The *purity* : $p=Tr(\hat{\rho}^2)$, which for a qubit is bounded as $1/2 \le p \le 1$. <br>
$\hat{\rho}$ is pure if $p=1$ and maximally mixed if $p=1/2$.

In this example, we select the target state by choosing a random Bloch vector and renormalizing it to have a specified purity.

To start, we import PennyLane, NumPy, and PyTorch for the optimization:

In [2]:
import pennylane as qml
from pennylane import numpy as np

Generate a three-dimensional random vector: $\vec{v}=(v_1,v_2,v_3)$ <br>
Purity: $p=0.66$

In [3]:
# we generate a three-dimensional random vector by sampling
# each entry from a standard normal distribution
v = np.random.normal(0, 1, 3)

# purity of the target state
purity = 0.66

Create a raondom Bloch vector with the specified purity: $\sqrt{2p -1} \times \frac{\vec{v}}{|\vec{v}|}$

In [4]:
# create a random Bloch vector with the specified purity
bloch_v = np.sqrt(2 * purity - 1) * v / np.sqrt(np.sum(v ** 2))

Pauli matrices

In [5]:
# array of Pauli matrices (will be useful later)
Paulis = np.zeros((3, 2, 2), dtype=complex)
Paulis[0] = [[0, 1], [1, 0]]
Paulis[1] = [[0, -1j], [1j, 0]]
Paulis[2] = [[1, 0], [0, -1]]

Unitary Oprators: pure states $\longrightarrow$ pure states. <br>

How to prepare mixed staes in quantum circuis? <br>
1. Introduce ancillary qubits and peform a unitary transformatino on this larger system.
2. By "tracing out"  the ancilla qubits, we can prepare mixed states in the target register. <br>

In this example, we introduce two additional qubits, which suffices to prepare arbitrary states.

**The ansatz circuit** is composed of repeated layers, each of which consists of single-qubit rotations along the $x$, $y$, and $z$ axes, followed by three CNOT gates entangling all qubits. <br>
Initial gate parameters are chosen at random from normal distribution. <br>
Importantly, when declaring the layer function, we introduce an input parameter $j$, which allows us to later call each layer individually.

In [6]:
# number of qubits in the circuit
n_qubits = 3
# number of layers in the circuit
n_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (n_qubits, n_layers, 3))

# a layer of the circuit ansatz
def layer(params, j):
    for i in range(n_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])

---
Select a plugin that is compatible with evaluating expectations through smapling: the `forest.qvm` plugin. <br>
The syntax is slightly different than for other plugins, we need to also feed a `device` keyword specifying the number of qubits in the format `[number of qubits]q-pyqvm`.<br>
The kyeworkd `shots` indicates the number of smaples used to estimate expectation values.


In [7]:
#dev = qml.device("forest.qvm", device="3q-pyqvm", shots=1000) # <-- I think this doesn't work anymore.
dev = qml.device("default.qubit", wires=3)

---

When defining the qnode, we introduce as input a Hermitian operator $A$ that specifies the expectation value being evaluated. <br>
This choice later allows us to easily evaluate several expectation values without having to define a new qnode each times.

In [8]:
@qml.qnode(dev)
def circuit(params, A=None):

    # repeatedly apply each layer in the circuit
    for j in range(n_layers):
        layer(params, j)

    # returns the expectation of the input matrix A on the first qubit
    return qml.expval(qml.Hermitian(A, wires=0))

Our goal is to prepare a state **with the same Bloch vector** as the target state. Therefore, we define a simple cost function
$$
C = \sum^{3}_{i=1} |a_i-v_i|,
$$

where $\vec{a}=(a_1,a_2,a_3)$ is the target vector and $\vec{v}=(v_1,v_2,v_3)$ is the vector of the state prepared by the circuit. <br>

In [9]:
# cost function
def cost_fn(params):
    cost = 0
    for k in range(3):
        cost += np.abs(circuit(params, A=Paulis[k]) - bloch_v[k])

    return cost

Optimization is carried out using the Adam optimizer. <br>
Finally, we compare the Bloch vectors of the target an output state.

In [11]:
# set up the optimizer
opt = qml.AdamOptimizer()

# number of steps in the optimization routine
steps = 200

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((n_qubits, n_layers, 3))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    params = opt.step(cost_fn, params)
    current_cost = cost_fn(params)

    # keeps track of best parameters
    if current_cost < best_cost:
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, current_cost))

Cost after 0 steps is 0.0102
Cost after 10 steps is 0.1630
Cost after 20 steps is 0.0233
Cost after 30 steps is 0.0073
Cost after 40 steps is 0.0192
Cost after 50 steps is 0.0143
Cost after 60 steps is 0.0141
Cost after 70 steps is 0.0093
Cost after 80 steps is 0.0116
Cost after 90 steps is 0.0025
Cost after 100 steps is 0.0027
Cost after 110 steps is 0.0036
Cost after 120 steps is 0.0055
Cost after 130 steps is 0.0043
Cost after 140 steps is 0.0017
Cost after 150 steps is 0.0087
Cost after 160 steps is 0.0084
Cost after 170 steps is 0.0009
Cost after 180 steps is 0.0031
Cost after 190 steps is 0.0048
Cost after 200 steps is 0.0031


In [12]:
# calculate the Bloch vector of the output state
output_bloch_v = np.zeros(3)
for l in range(3):
    output_bloch_v[l] = circuit(best_params, A=Paulis[l])

# print results
print("Target Bloch vector = ", bloch_v)
print("Output Bloch vector = ", output_bloch_v)

Target Bloch vector =  [ 0.03574474  0.26410515 -0.49896972]
Output Bloch vector =  [ 0.03586431  0.26126027 -0.49879198]


---
### Note

A pure state $|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\phi}\sin(\theta/2)|1\rangle$, where $0\le\theta\le\pi$  and $0\le\phi\le 2\pi$.


Block vector $\vec{a}=(\sin \theta \cos \phi, \sin \theta \sin \phi, \cos \theta)$.